In [ ]:
!pip install pillow

In [1]:
import numpy as np
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt 
%matplotlib inline


from glob import glob
import os 

from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold, StratifiedKFold

from sklearn.preprocessing import StandardScaler

from sklearn.metrics import accuracy_score

from imblearn.over_sampling import SMOTE

import torch
from torch import nn, optim
import timm

import random

from PIL import Image

import albumentations as A
from albumentations.pytorch import ToTensorV2

from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

from tqdm import tqdm 

device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [2]:
train = pd.read_csv('mnist/train.csv')
test = pd.read_csv('mnist/test.csv')
submission = pd.read_csv('mnist/sample_submission.csv')

In [3]:
def seed_everything(seed) :
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    

def NMAE(true, pred) : 
    mae = np.mean(np.abs(true-pred))
    score = mae/np.mean(np.abs(true))
    return score

SEED = 42
seed_everything(SEED)

In [4]:
IMAGE_SIZE = 28
TEST_SIZE = 0.2
N_EPOCH = 5
LR = 1e-3
BATCH_SIZE = 32
MODEL_NAME = 'densenet121'

# Dataset

In [5]:
class train_dataset(Dataset) :
    def __init__(self, train_df, transform=None ) :
        super(train_dataset).__init__()
        self.train_df = train_df.drop(['index', 'label'],1)
        self.target = train_df.label
        self.transform = transform 
        
    def __len__(self) : 
        return len(self.train_df)
    
    def __getitem__(self, idx) : 
        img = self.train_df.to_numpy()[idx].reshape(28,28)
        img = torch.tensor(img/255).view(1,28,28)
        img = img.type(torch.float32)
        
        #if self.transform : 
        #    img = self.transform(image=img)['image']
        
        target = self.target[idx]
        target = torch.tensor(target, dtype=torch.float32)
        
        return img, target

In [6]:
class test_dataset(Dataset) : 
    def __init__(self, test_imgs, transform =None) :
        super(test_dataset).__init__() 
        self.test_imgs = test_imgs.drop(['index'], 1)
        self.transform = transform
        
    def __len__(self) :
        return len(self.test_imgs)
    
    def __getitem__(self, idx) :  
        img = self.test_imgs.to_numpy()[idx].reshape(28,28)
        img = torch.tensor(img/255).view(1,28,28)
        img = img.type(torch.float32)        
        #if self.transform : 
        #    img = self.transform(image=img)['image']
            
        return img
        

In [7]:
img_transform = A.Compose([
    A.Resize(IMAGE_SIZE, IMAGE_SIZE), 
    A.Normalize(),
    ToTensorV2()
])

# DataLoader 

In [8]:
train_loader = DataLoader(train_dataset(train), batch_size = BATCH_SIZE, shuffle=True)

In [9]:
test_loader = DataLoader(test_dataset(test), batch_size = BATCH_SIZE, shuffle=False)

# Model

In [10]:
class ConvNet(nn.Module) : 
    def __init__(self) : 
        super(ConvNet, self).__init__()
        self.layer1 = torch.nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))
        
        self.layer2 = torch.nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2))      
        
        self.fc = nn.Linear(7*7*64,10)
        
        
    def forward(self, x) : 
        # (batch, 3, image_size, image_size)-> batch, 64, resize, resize)
        # 128*128 -> 7*7
        x = self.layer1(x)
        x = self.layer2(x)
        
        x = torch.flatten(x, start_dim=1)
        out = self.fc(x)
        return out

In [11]:
model = ConvNet().to(device)

In [12]:
criterion = torch.nn.CrossEntropyLoss().to(device)    # 비용 함수에 소프트맥스 함수 포함되어져 있음.
optimizer = torch.optim.Adam(model.parameters(), lr=LR)

# Train/Validation

In [16]:
weight_path = 'MNIST_Classification_CNN_Model.pt'

In [25]:
train_df, val_df = train_test_split(train, test_size = 0.2, random_state = SEED, stratify = train['label'])
train_df = train_df.reset_index(drop=True)
val_df = val_df.reset_index(drop=True)

In [26]:
train_loader = DataLoader(train_dataset(train_df), shuffle= True, batch_size = BATCH_SIZE)
val_loader = DataLoader(train_dataset(val_df), shuffle= False, batch_size = BATCH_SIZE)

In [19]:
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', patience=2, factor = 0.5, min_lr = 5e-5)

In [14]:
from sklearn.metrics import accuracy_score

In [28]:
val_score = 0

for epoch in range(N_EPOCH):
    avg_cost = 0

    for X, Y in train_loader: # 미니 배치 단위로 꺼내온다. X는 미니 배치, Y는 레이블.
        # image is already size of (28x28), no reshape
        # label is not one-hot encoded
        Y = Y.type(torch.LongTensor)
        X = X.to(device)
        Y = Y.to(device)

        optimizer.zero_grad()
        hypothesis = model(X)
        cost = criterion(hypothesis, Y)
        cost.backward()
        optimizer.step()

        avg_cost += cost / len(train_loader)

    print('[Epoch: {:>4}] cost = {:>.9}'.format(epoch + 1, avg_cost))
    
    model.eval()
    with torch.no_grad() :
        preds = [] 
        val_loss = 0
        for X, Y in val_loader : 
            Y = Y.type(torch.LongTensor)
            X = X.to(device)
            Y = Y.to(device)
            hypothesis = model(X)
            batch_cost = criterion(hypothesis, Y)
            val_loss += batch_cost / len(val_loader)
            
            pred = hypothesis.cpu().numpy()
            preds.append(np.argmax(pred,1))
            
        accuracy = accuracy_score(val_df.label, np.concatenate(preds))
        
        if val_score < accuracy : 
            print(f'val_accuracy improved {val_score:.4f} to {accuracy:.4f}')
            val_score = accuracy 
            torch.save(model.state_dict(), weight_path)
        print(f'[Epoch {epoch+1} / {N_EPOCH}] cost = {avg_cost:.4f}, val_accuracy={val_score:.4f}')
        
model.load_state_dict(torch.load(weights_path))
            
            

[Epoch:    1] cost = 0.0125698429
val_accuracy improved 0.0000 to 0.9926
[Epoch 1 / 5] cost = 0.0126, val_accuracy=0.9926
[Epoch:    2] cost = 0.0106809167
[Epoch 2 / 5] cost = 0.0107, val_accuracy=0.9926
[Epoch:    3] cost = 0.00828756113
[Epoch 3 / 5] cost = 0.0083, val_accuracy=0.9926
[Epoch:    4] cost = 0.00633600075
[Epoch 4 / 5] cost = 0.0063, val_accuracy=0.9926
[Epoch:    5] cost = 0.0059860223
[Epoch 5 / 5] cost = 0.0060, val_accuracy=0.9926


NameError: name 'weights_path' is not defined

# Inference

In [ ]:
preds = []
model.eval()
with torch.no_grad() : 
    
    for X in tqdm(test_loader) : 
        X = X.to(device)
        pred = model(X)
        pred = pred.cpu().numpy()
        preds.append(np.argmax(pred,1))
        

In [ ]:
preds_array = np.concatenate(preds)

In [ ]:
submission['label_pred'] = preds_array

In [ ]:
submission['result'] = submission['label'] - submission['label_pred']

In [ ]:
submission['result'].sum()